## Instalando ultralytics

In [ ]:
!pip install ultralytics

## Importando requisitos

> ### Adicionando lib ao path do sistema para scripts serem importados

In [ ]:
!git clone https://github.com/andrac-23/projeto-mc906.git repositorio_projeto

In [ ]:
import sys
sys.path.append("/kaggle/working/repositorio_projeto/lib/")

In [ ]:
import os
import torch
import gc
from IPython.display import clear_output, display, Image
import ultralytics
from ultralytics import YOLO
import shutil
import pandas as pd
import proj_mc906_plotter as proj_plotter

## Limpando saídas e usando o Garbage Collector

In [ ]:
clear_output()
gc.collect()

## Verificando se a GPU está disponível
> ### Se não estiver, verificar no tutorial para ativá-la

In [ ]:
if torch.cuda.is_available():
    print("GPU disponível")
    print(torch.cuda.memory_allocated())
else:
    raise RuntimeError("GPU não está disponível. Saindo...")

## Copiar o dataset para o working directory
### O diretório `../input/dataset` é read-only apenas. Verifique se o print está no formato esperado (pastas `train/val/test` e arquivo `data.yaml`)

In [ ]:
erro: descomentar o path do dataset atual
# ================= v3_2_M ====================== #
# input_dataset_path = "../input/dataset-v3-2/v3_M_yoloV8_files"
# dataset_path = "/kaggle/working/dataset-v3-2"
# ================= v2_P ======================== #
# input_dataset_path = "../input/v2-p-recognition"
# dataset_path = "/kaggle/working/v2-p-recognition"
# =============================================== #

# copiar para o working directory
shutil.copytree(input_dataset_path, dataset_path)
# verificar se o print está certo
print(os.listdir(dataset_path))

## Definindo e criando diretórios

In [ ]:
data_yaml_path = os.path.join(dataset_path, "data.yaml")
dataset_name = os.path.basename(dataset_path)
results_path = f"/kaggle/working/{dataset_name}_yoloV8_results/"
training_path = os.path.join(results_path, "train")
os.makedirs(os.path.join(results_path, "final_weights"), exist_ok=True)
results_table = None
plots_path = os.path.join(results_path, "plots")
os.makedirs(plots_path, exist_ok=True)

## Treinando modelo

### Documentação: [link](https://docs.ultralytics.com/modes/train/#train-settings)

In [ ]:
ultralytics.checks()

In [ ]:
erro: conferir a letra da versão tamanho_yolo
    
MAX_EPOCHS = 300
automatico = True # resultados automaticos
tamanho_yolo = "s" # inserir letra da versão (n: nano, s: small, m: medium)
model_arguments = {
    "patience": 10, # comentar essa linha se automatico virar False
    "project": results_path,
    "data": data_yaml_path,
    "batch": 64,
    "epochs": 10#,
    # parametros adicionais
    #"optimizer": 
    #"lr0": 
    #"momentum":
    # etc
}

In [ ]:
tot_epochs, best_mAP50 = 0, -1
best_results = {}
while tot_epochs <= MAX_EPOCHS:
    model = YOLO(f"yolov8{tamanho_yolo}.pt") # modelo inicial
    if tot_epochs > 0:
        # carregar o modelo já treinado
        last_weight_path = os.path.join(results_path, "final_weights", "last.pt")
        model = YOLO(last_weight_path)
        model.resume = True

    # treinar o modelo
    results = model.train(**model_arguments)
    tot_epochs += model_arguments["epochs"] # adicionar épocas treinadas
    
    if automatico:
        best_results = results
        os.rename(os.path.join(training_path), os.path.join(results_path, "resultados_automaticos"))
        os.rmdir(os.path.join(results_path, "plots"))
        os.rmdir(os.path.join(results_path, "final_weights"))
        break
    
    # abrir results.csv do treinamento atual como "table" e adicionar resultados em "results_table"
    table = pd.read_csv(os.path.join(training_path, "results.csv"))
    table.columns = table.columns.str.strip()
    if results_table is not None:
        table["epoch"] = range(len(results_table) + 1, len(results_table) + len(table) + 1)
        results_table = pd.concat([results_table, table], ignore_index=True)
    else:
        results_table = table
    # salvar results_table novo
    results_table.to_csv(os.path.join(results_path, "results.csv"), index=False)
    
    # salvar pesos atuais
    atualizado = False
    if best_mAP50 < results.results_dict["metrics/mAP50(B)"]:
        # atualiza best.pt
        atualizado = True
        best_results = results
        best_mAP50 = results.results_dict["metrics/mAP50(B)"]
        os.rename(os.path.join(training_path, "weights", "best.pt"), os.path.join(results_path, "final_weights", "best.pt"))
    os.rename(os.path.join(training_path, "weights", "last.pt"), os.path.join(results_path, "final_weights", "last.pt"))
    
    # plotar os gráficos de perdas
    curr_starting_epoch, curr_ending_epoch = table["epoch"].iloc[0], table["epoch"].iloc[-1]
    plots_path = os.path.join(results_path, "plots", f"1-{curr_ending_epoch}_loss_plots.png")
    proj_plotter.create_loss_plots(results_table, plots_path)
    # mostrar os gráficos de perdas
    clear_output(wait=True)
    if atualizado:
        print("Best.pt foi atualizado com novos melhores pesos.\n")
    print("Mostrando gráficos de perdas para análises:")
    display(Image(filename=plots_path))
    
    # remover pasta de treino antiga
    shutil.rmtree(os.path.join(training_path))
    
    continue_training = input("Continuar treinamento? (s/n): ")
    if continue_training.lower() != "s":
        print("Treinamento finalizado")
        break
    while True:
        current_op = input("Deseja ajustar os hiperparâmetros?:\nb - batch size\nl - learning rate\ne - epochs\ns - salvar e sair\n")
        if current_op == "s":
            print(f"\nResumindo treinamento por mais {model_arguments['epochs']} epochs...\n")
            break
        elif current_op == "b":
            pass
        elif current_op == "l":
            pass
        elif current_op == "e":
            pass
        else:
            print("Operação inválida!")

## Escrevendo resultados finais do treinamento (métricas da validação)

In [ ]:
with open(os.path.join(results_path, "resumo.txt"), "w") as f:
    f.write(f"Dataset Treinado: {dataset_name}\n")
    f.write(f"Modelo Treinado: YOLOv8 ({tamanho_yolo})\n\n")
    f.write(f"Parâmetros customizados:\n")
    for key, val in model_arguments.items():
        if key == "project" or key == "data":
            continue
        elif key == "epochs":
            f.write(f"{key}: {tot_epochs}\n")
        else:
            f.write(f"{key}: {val}\n")
    f.write("\nResultados finais de treinamento (métricas da validação com best.pt):\n")
    for key, val in best_results.results_dict.items():
        f.write(f"{key}: {val}\n")
    

## Validar modelo no teste
### Documentação (com parâmetros): [link](https://docs.ultralytics.com/modes/val/)

In [ ]:
best_path = os.path.join(results_path, "final_weights", "best.pt")
if automatico:
    best_path = os.path.join(results_path, "resultados_automaticos", "weights", "best.pt")
    
model = YOLO(best_path)
results = model.val(data=data_yaml_path, plots=True, batch=model_arguments["batch"], split="test")  # testar a performance do best.pt no test set

In [ ]:
os.makedirs(os.path.join(results_path, "test_results"), exist_ok=True)
os.rename(os.path.join("/kaggle", "working", "runs", "detect", "val"), os.path.join(results_path, "test_results"))
shutil.rmtree(os.path.join("/kaggle", "working", "runs"))

## Escrevendo resultados finais da validação com o split de teste

In [ ]:
with open(os.path.join(results_path, "test_results", "resultados.txt"), "w") as f:
    f.write(f"Dataset Treinado: {dataset_name}\n")
    f.write(f"Modelo Treinado: YOLOv8 ({tamanho_yolo})\n\n")
    f.write("Resultados finais da validação com o split teste usando o best.pt:\n")
    for key, val in results.results_dict.items():
        f.write(f"{key}: {val}\n")

## Salvando os resultados
### Após o comando ser executado, procure no lado direito o arquivo `.zip` criado, clique nos três pontos verticais e faça o download

In [ ]:
erro: descomentar o dataset utilizado
# ================= v3_2_M ====================== #
#!zip -r dataset-v3-2_yoloV8_results.zip dataset-v3-2_yoloV8_results
# ================= v2_P ======================== #
#!zip -r v2-p-recognition_yoloV8_results.zip v2-p-recognition_yoloV8_results
